In [31]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import optuna
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import VotingClassifier

In [32]:
df = pd.read_csv('youtoxic_english_1000.csv')

# Eliminar las columnas 'CommentId' y 'VideoId'
df = df.drop(columns=['CommentId', 'VideoId', 'IsSexist', 'IsHomophobic', 'IsRadicalism'])

# Mostramos las primeras filas y la información del dataset
print("Primeras filas del dataset:")
display(df.head())
print("\nInformación del dataset:")
display(df.info())

Primeras filas del dataset:


,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsReligiousHate
0,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False
1,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False
2,\r\nDont you reckon them 'black lives matter' ...,True,True,False,False,True,False,False,False,False
3,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False
4,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False



Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Text             1000 non-null   object
 1   IsToxic          1000 non-null   bool  
 2   IsAbusive        1000 non-null   bool  
 3   IsThreat         1000 non-null   bool  
 4   IsProvocative    1000 non-null   bool  
 5   IsObscene        1000 non-null   bool  
 6   IsHatespeech     1000 non-null   bool  
 7   IsRacist         1000 non-null   bool  
 8   IsNationalist    1000 non-null   bool  
 9   IsReligiousHate  1000 non-null   bool  
dtypes: bool(9), object(1)
memory usage: 16.7+ KB


None

In [33]:
# [Celda 3] - Preparación y limpieza de datos

# Primero mostramos información sobre valores nulos en el dataset
print("Valores nulos por columna antes de la limpieza:")
print(df.isnull().sum())
print("\nTotal de filas antes de la limpieza:", len(df))

# Definimos las columnas que indican contenido dañino
toxic_columns = ['IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 
                'IsObscene', 'IsHatespeech', 'IsRacist', 'IsNationalist',
                'IsReligiousHate']

# Convertir los valores "TRUE"/"FALSE" a 1/0 en las columnas tóxicas (si es necesario)
df[toxic_columns] = df[toxic_columns].replace({'TRUE': 1, 'FALSE': 0})

# Creamos una nueva columna que será 1 si cualquiera de las categorías es 1
df['is_harmful'] = df[toxic_columns].any(axis=1).astype(int)

# Preparamos X (textos) e y (etiquetas)
X = df['Text']
y = df['is_harmful']

# Mostramos estadísticas finales
print("\nEstadísticas finales:")
print("-" * 50)
print("Distribución de mensajes dañinos vs no dañinos:")
print(y.value_counts(normalize=True))
print("\nDistribución por tipo de contenido dañino:")
for col in toxic_columns:
    positivos = df[col].sum()
    porcentaje = (positivos / len(df)) * 100
    print(f"{col}: {positivos} casos ({porcentaje:.2f}%)")

Valores nulos por columna antes de la limpieza:
Text               0
IsToxic            0
IsAbusive          0
IsThreat           0
IsProvocative      0
IsObscene          0
IsHatespeech       0
IsRacist           0
IsNationalist      0
IsReligiousHate    0
dtype: int64

Total de filas antes de la limpieza: 1000

Estadísticas finales:
--------------------------------------------------
Distribución de mensajes dañinos vs no dañinos:
is_harmful
0    0.538
1    0.462
Name: proportion, dtype: float64

Distribución por tipo de contenido dañino:
IsToxic: 462 casos (46.20%)
IsAbusive: 353 casos (35.30%)
IsThreat: 21 casos (2.10%)
IsProvocative: 161 casos (16.10%)
IsObscene: 100 casos (10.00%)
IsHatespeech: 138 casos (13.80%)
IsRacist: 125 casos (12.50%)
IsNationalist: 8 casos (0.80%)
IsReligiousHate: 12 casos (1.20%)


In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.15,
    random_state=42,
    stratify=y  # Mantenemos la proporción de clases
)

print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de prueba:", len(X_test))

Tamaño del conjunto de entrenamiento: 850
Tamaño del conjunto de prueba: 150


In [35]:
vectorizer_1 = TfidfVectorizer(
    max_features=5000,  # Limitamos el número de características
    min_df=2,          # Ignoramos términos que aparecen en menos de 2 documentos
    stop_words='english'  # Removemos stopwords en inglés
)

# Transformamos los textos
X_train_vectorized_1 = vectorizer_1.fit_transform(X_train)
X_test_vectorized_1 = vectorizer_1.transform(X_test)

print("Dimensiones de la matriz de características de entrenamiento:", X_train_vectorized_1.shape)

Dimensiones de la matriz de características de entrenamiento: (850, 1566)


In [36]:
final_svm_model = SVC(
    C=109.42844096035671,
    kernel='rbf',
    gamma=1.1662316858478285,
    probability=True,
    random_state=42,
    class_weight=None,
    tol=0.00043366566542372227,
    shrinking=False,
    max_iter=1697
)

# Entrenar el modelo con el conjunto de entrenamiento completo
final_svm_model.fit(X_train_vectorized_1, y_train)

print("Modelo final entrenado con los mejores hiperparámetros!")

Modelo final entrenado con los mejores hiperparámetros!


In [37]:
import optuna
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# Definir la función objetivo para Optuna con MultinomialNB
def objective_nb(trial):
    # Definir los hiperparámetros a optimizar
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)  # Parámetro de suavizado
    fit_prior = trial.suggest_categorical('fit_prior', [True, False])  # Ajustar o no las probabilidades previas
    
    # Crear el modelo Naive Bayes con los hiperparámetros seleccionados
    nb_model = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
    
    # Evaluar el modelo con validación cruzada
    scores = cross_val_score(nb_model, X_train_vectorized_1, y_train, cv=3, scoring='accuracy')
    accuracy = scores.mean()  # Promedio de accuracy
    
    return accuracy  # Optuna maximiza este valor

# Crear un estudio de optimización para MultinomialNB
study_nb = optuna.create_study(direction='maximize')
study_nb.optimize(objective_nb, n_trials=50)  # Ajusta el número de pruebas según el tiempo disponible

# Resultados de la mejor prueba para MultinomialNB
print("Mejores hiperparámetros para MultinomialNB:")
print(study_nb.best_trial.params)
print("Mejor precisión:", study_nb.best_trial.value)

[I 2024-11-12 14:45:48,681] A new study created in memory with name: no-name-40dd0288-30f4-4a70-9a31-5cdecdf6b894
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9652\4272701593.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2)  # Parámetro de suavizado
[I 2024-11-12 14:45:48,701] Trial 0 finished with value: 0.6635436885150716 and parameters: {'alpha': 0.09347877541058479, 'fit_prior': False}. Best is trial 0 with value: 0.6635436885150716.
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9652\4272701593.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-5, 1e2) 

Mejores hiperparámetros para MultinomialNB:
{'alpha': 1.664189892739271, 'fit_prior': True}
Mejor precisión: 0.6929569170026045


In [38]:
# Extraer los mejores hiperparámetros del estudio de MultinomialNB
best_params_nb = study_nb.best_trial.params

# Crear el modelo final con los mejores hiperparámetros
final_nb_model = MultinomialNB(
    alpha=best_params_nb['alpha'],
    fit_prior=best_params_nb['fit_prior']
)

# Entrenar el modelo con el conjunto de entrenamiento completo
final_nb_model.fit(X_train_vectorized_1, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred_nb = final_nb_model.predict(X_test_vectorized_1)

# Calcular y mostrar la precisión y el reporte de clasificación
from sklearn.metrics import accuracy_score, classification_report
accuracy_nb = accuracy_score(y_test, y_pred_nb)
report_nb = classification_report(y_test, y_pred_nb)

print("Accuracy en conjunto de prueba para MultinomialNB:", accuracy_nb)
print("Reporte de clasificación para MultinomialNB:\n", report_nb)

Accuracy en conjunto de prueba para MultinomialNB: 0.72
Reporte de clasificación para MultinomialNB:
               precision    recall  f1-score   support

           0       0.70      0.85      0.77        81
           1       0.76      0.57      0.65        69

    accuracy                           0.72       150
   macro avg       0.73      0.71      0.71       150
weighted avg       0.73      0.72      0.71       150



In [39]:
from sklearn.ensemble import RandomForestClassifier

final_rf_model = RandomForestClassifier(
    n_estimators=235,
    max_depth=44,
    min_samples_split=10,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42
)

# Entrenar el modelo con el conjunto de entrenamiento completo
final_rf_model.fit(X_train_vectorized_1, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred_rf = final_rf_model.predict(X_test_vectorized_1)

# Calcular y mostrar la precisión y el reporte de clasificación
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)

print("Accuracy en conjunto de prueba para RandomForestClassifier:", accuracy_rf)
print("Reporte de clasificación para RandomForestClassifier:\n", report_rf)


Accuracy en conjunto de prueba para RandomForestClassifier: 0.7066666666666667
Reporte de clasificación para RandomForestClassifier:
               precision    recall  f1-score   support

           0       0.67      0.91      0.77        81
           1       0.82      0.46      0.59        69

    accuracy                           0.71       150
   macro avg       0.74      0.69      0.68       150
weighted avg       0.74      0.71      0.69       150



In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import sparse

# 1. Primero dividimos los datos en conjuntos de entrenamiento y prueba
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.01,
    random_state=42,
    stratify=y
)

# Separamos el conjunto temporal en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.1,
    random_state=42,
    stratify=y_temp
)

# 2. Vectorizamos los datos
vectorizer_2 = TfidfVectorizer(
    min_df=2,
    max_df=0.95,
    max_features=5000
)

# Vectorizamos cada conjunto por separado
X_train_vectorized_2 = vectorizer_2.fit_transform(X_train)
X_val_vectorized = vectorizer_2.transform(X_val)
X_test_vectorized_2 = vectorizer_2.transform(X_test)

# Reducir dimensionalidad
selector = SelectKBest(score_func=chi2, k=1000)
X_train_reduced = selector.fit_transform(X_train_vectorized_2, y_train)
X_val_reduced = selector.transform(X_val_vectorized)
X_test_reduced = selector.transform(X_test_vectorized_2)

print("Dimensiones después de la reducción:")
print("Entrenamiento:", X_train_reduced.shape)
print("Validación:", X_val_reduced.shape)
print("Prueba:", X_test_reduced.shape)

# Integración de MLflow
import mlflow
mlflow.set_experiment("my-experiment")

with mlflow.start_run():
    mlflow.log_params({
        "max_depth": 5,
        "n_estimators": 100,
        "learning_rate": 0.1
    })

    # 4. Creamos y entrenamos el ensemble con bagging
    bagged_ensemble = BaggingClassifier(
        estimator=VotingClassifier(
            estimators=[
                ('random_forest', final_rf_model),
                ('naive_bayes', final_nb_model),
                ('svm', final_svm_model)
            ],
            voting='soft'
        ),
        n_estimators=10,
        max_samples=0.8,
        max_features=0.8,
        random_state=42
    )

    # 5. Implementamos early stopping
    class EarlyStoppingMonitor:
        def __init__(self, patience=3, min_delta=0.001):
            self.patience = patience
            self.min_delta = min_delta
            self.best_score = None
            self.counter = 0
            self.best_model = None
            
        def check(self, model, val_score):
            if self.best_score is None or val_score > self.best_score + self.min_delta:
                self.best_score = val_score
                self.counter = 0
                self.best_model = model
                return False
            else:
                self.counter += 1
                return self.counter >= self.patience

    # Entrenamiento con early stopping
    monitor = EarlyStoppingMonitor(patience=3)
    best_accuracy = 0

    for epoch in range(10):
        # Entrenar el modelo
        bagged_ensemble.fit(X_train_reduced, y_train)
        
        # Evaluar en el conjunto de validación
        val_accuracy = bagged_ensemble.score(X_val_reduced, y_val)
        print(f"Época {epoch + 1}, Accuracy en validación: {val_accuracy:.4f}")
        
        if monitor.check(bagged_ensemble, val_accuracy):
            print("Early stopping activado en época", epoch + 1)
            break

    # Usar el mejor modelo encontrado
    best_model = monitor.best_model if monitor.best_model is not None else bagged_ensemble


Dimensiones después de la reducción:
Entrenamiento: (891, 1000)
Validación: (99, 1000)
Prueba: (10, 1000)


C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\

Época 1, Accuracy en validación: 0.7778


C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\

Época 2, Accuracy en validación: 0.7778


C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\

Época 3, Accuracy en validación: 0.7778


C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1697).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\Administrator\AppData\

Época 4, Accuracy en validación: 0.7778
Early stopping activado en época 4


In [47]:
# Evaluar el mejor modelo con threshold
threshold = 0.59

# Predicciones en entrenamiento
y_prob_train = best_model.predict_proba(X_train_reduced)[:, 1]
y_pred_train_thresholded = (y_prob_train >= threshold).astype(int)

# Predicciones en prueba
y_prob_test = best_model.predict_proba(X_test_reduced)[:, 1]
y_pred_test_thresholded = (y_prob_test >= threshold).astype(int)

# Calcular métricas
accuracy_train = accuracy_score(y_train, y_pred_train_thresholded)
accuracy_test = accuracy_score(y_test, y_pred_test_thresholded)
overfitting_percentage = ((accuracy_train - accuracy_test) / accuracy_train) * 100

# Imprimir resultados
print("\nResultados finales:")
print(f"Accuracy en entrenamiento (threshold={threshold}): {accuracy_train:.4f}")
print(f"Accuracy en prueba (threshold={threshold}): {accuracy_test:.4f}")
print(f"Porcentaje de overfitting: {overfitting_percentage:.2f}%")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_test_thresholded))

# Visualizar las características más importantes
if hasattr(selector, 'get_feature_names_out'):
    feature_names = vectorizer_2.get_feature_names_out()
    selected_features = selector.get_support()
    important_features = [(feature_names[i], selector.scores_[i]) 
                         for i in range(len(feature_names)) 
                         if selected_features[i]]
    
    important_features.sort(key=lambda x: x[1], reverse=True)
    print("\nTop 10 características más importantes:")
    for feature, score in important_features[:10]:
        print(f"{feature}: {score:.4f}")


Resultados finales:
Accuracy en entrenamiento (threshold=0.59): 0.8013
Accuracy en prueba (threshold=0.59): 0.8000
Porcentaje de overfitting: 0.17%

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       1.00      0.60      0.75         5

    accuracy                           0.80        10
   macro avg       0.86      0.80      0.79        10
weighted avg       0.86      0.80      0.79        10


Top 10 características más importantes:
run: 8.1457
fuck: 7.1270
over: 6.6136
them: 5.8197
idiot: 5.2509
shit: 5.2149
ass: 5.1603
peggy: 4.8790
fucking: 4.6901
bitch: 4.6859


In [48]:
import joblib

# Guardar el vectorizador
joblib.dump(vectorizer_2, 'vectorizer_2.pkl')

# Guardar el selector de características
joblib.dump(selector, 'feature_selector.pkl')

# Guardar el modelo entrenado
joblib.dump(best_model, 'ensemble_model.pkl')


['ensemble_model.pkl']